Please enter the two cases below

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from numpy.linalg import inv
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import sklearn.metrics.pairwise as dist
from tqdm import tqdm
import seaborn as sns
%matplotlib inline
ratings=pd.read_csv('../../new_dataset/action_movies.csv',index_col=0)
movies_titles=pd.read_csv("../../new_dataset/action_titles.csv")
df_ref=pd.read_csv('../../new_dataset/correspondances_Id_movie')

In [18]:


user = input("Type a number between 0 and 1167 to indicate your IdUser : ")
movies_asked= input("Type the number of movies you want to watch : ")
user=int(user)
movies_asked=int(movies_asked)


n_users = ratings.Id_Action.unique().shape[0]
n_movies = ratings.movie_Action.unique().shape[0]

filtre=list(ratings["movie_Action"][ratings["Id_Action"]==user].unique())

films_vus_par_u = ratings[ratings["movie_Action"].isin(filtre)]

#In a first we suppose assume that there is no user to remove but only movies to reindex
n_users=len(films_vus_par_u["Id_Action"].unique())
n_movies=len(films_vus_par_u["movie_Action"].unique())


jointure=pd.merge(movies_titles,ratings)
jointure.drop(["movieId_ref","userId","Id_Action","rating"],axis=1,inplace=True)
jointure.head()
jointure.head()

movies_seen=jointure[jointure["movie_Action"].isin(filtre)]
print("Bravo, you have already seen ",str(n_movies) + " movies")
print("The list of movies I advice you is below")
print("this is the list of movies already seen : ",list(movies_seen["title"].unique()))

films_vus_par_u.drop(["userId","movieId_ref"],axis=1,inplace=True)

conversion_Id_movies_seen=pd.DataFrame({"movie_Action":list(films_vus_par_u["movie_Action"].unique()),"new_movies_Id":range(0,n_movies)})
result=pd.merge(films_vus_par_u, conversion_Id_movies_seen, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

matrice=np.zeros(( n_users,n_movies))
for line in result.itertuples():
    matrice[line[1], line[4]]=line[2]

user_similarity=dist.cosine_similarity(matrice)

a=list(ratings["movie_Action"].unique())
b=list(movies_seen["movie_Action"].unique())
movie_no_seen=[item for item in a if item not in b]
movie_no_seen=pd.DataFrame({"movie_Action" : movie_no_seen})
movie_no_seen=pd.merge(movie_no_seen,ratings,how='inner')

n_movies=len(movie_no_seen.movie_Action.unique())
#Il faut construire un nouveau index des Id_movies
conversion_Id_movies_no_seen=pd.DataFrame({"movie_Action":list(movie_no_seen["movie_Action"].unique()),"new_movies_Id":range(0,n_movies)})

result=pd.merge(movie_no_seen, conversion_Id_movies_no_seen, how='inner', on=None, left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

matrice_new=np.zeros(( 1168,n_movies))
for line in result.itertuples():
    matrice_new[line[3], line[6]]=line[5]
    
def pred_user(matrice_modele, user_similarity, k, user):    
    pred = np.zeros(matrice_modele.shape[1])
    top_k_users = np.argsort(user_similarity[:,user])[-1:-k-1:-1]  
    for i in tqdm(range(matrice_modele.shape[1])):
        pred[i]=user_similarity[user,:][top_k_users].dot(matrice_modele[:,i][top_k_users])
        pred[i]/=np.sum(np.abs(user_similarity[user,:][top_k_users]))+0.000001     
    return pred 

movies_adviced=np.argsort(pred_user(matrice_new, user_similarity, 15, user))[-1:-2:-1]
movies_adviced=list(result["movie_Action"][result["new_movies_Id"].isin(movies_adviced)].unique())
movies_adviced=list(ratings["movieId_ref"][ratings["movie_Action"].isin(movies_adviced)].unique())

movies_adviced=list(movies_titles["title"][ratings["movieId_ref"].isin(movies_adviced)].unique())

print("Here are movies you will love")
movies_adviced[::-1][-1-movies_asked:-1]

Type a number between 0 and 1167 to indicate your IdUser :  1167
Type the number of movies you want to watch :  100


Bravo, you have already seen  344 movies
The list of movies I advice you is below
this is the list of movies already seen :  ['Net, The (1995)', 'Just Cause (1995)', 'Star Wars: Episode IV - A New Hope (1977)', 'Stargate (1994)', 'Maverick (1994)', 'True Lies (1994)', 'Beverly Hills Cop III (1994)', 'Three Musketeers, The (1993)', 'Terminator 2: Judgment Day (1991)', 'Wild Bunch, The (1969)', 'Twister (1996)', 'Spy Hard (1996)', 'Independence Day (a.k.a. ID4) (1996)', 'Die Hard (1988)', 'Bonnie and Clyde (1967)', 'Top Gun (1986)', 'Escape from New York (1981)', 'Star Wars: Episode V - The Empire Strikes Back (1980)', 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)', 'Aliens (1986)', 'Star Wars: Episode VI - Return of the Jedi (1983)', 'Terminator, The (1984)', 'Femme Nikita, La (Nikita) (1990)', 'Highlander (1986)', 'High Noon (1952)', 'Indiana Jones and the Last Crusade (1989)', 'Forbidden Planet (1956)', 'Alien³ (a.k.a. Alien 3) (1992)', 'Star Trek V: 

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
100%|██████████| 717/717 [00:00<00:00, 18517.96it/s]

Here are movies you will love


['Falcon Rising (2014)',
 'Stay (2013)',
 'Omega Doom (1996)',
 'Chair, The (2007)',
 'Gamera vs. Guiron (1969)',
 'Dallas Buyers Club (2013)',
 'Kon-Tiki (2012)',
 'Cuckoo in a Dark Forest (Kukacka v temném lese) (1986) ',
 'Romantics, The (2010)',
 'Expendables, The (2010)',
 'Downloading Nancy (2008)',
 'Hustle (1975)',
 'Big Stampede, The (1932)',
 'Sound of the Mountain (Thunder of the Mountain, The) (Yama no oto) (1954)',
 'Motel, The (2005)',
 'Five Pennies, The (1959)',
 'King Kong (2005)',
 'Simon (2004)',
 'Handful of Dust, A (1988)',
 'Samurai I: Musashi Miyamoto (Miyamoto Musashi) (1954)',
 'Executive Suite (1954)',
 'Die, Mommie, Die (2003)',
 'Women in Love (1969)',
 'Interview with the Assassin (2002)',
 'Used Cars (1980)',
 'Gentlemen Prefer Blondes (1953)',
 "Jacob's Ladder (1990)",
 'Excalibur (1981)',
 'Simply Irresistible (1999)',
 'Fried Green Tomatoes (1991)']